In [1]:
import numpy as np

lx = 18.0 # Armstrong
ly = 18.0 # Armstrong
lz = 7.40 # Armstrong
quitar = 1.8

molar_mass_AlCl3 =  133.34 # g/mol
density_AlCl3 = 2.48 * 1/(10**24)#g/A^3

molar_mass_EMIm_Cl = 146.62 # g/mol
density_EMIm_Cl = 1.189* 1/(10**24) #g/A^3



molar_mass_AlCl3_weighted = 10*molar_mass_AlCl3
mass_EMIm_Cl_weighted = 13*molar_mass_EMIm_Cl

volume =  lx*ly*lz  #(lx-1)*(ly-1)*(lz-quitar)

vol_AlCl3 = molar_mass_AlCl3_weighted/density_AlCl3
vol_EMIm_Cl = mass_EMIm_Cl_weighted/density_EMIm_Cl

total_vol = vol_AlCl3 + vol_EMIm_Cl

ratio_vv_AlCl3 = vol_AlCl3/total_vol
ratio_vv_EMIm_Cl = vol_EMIm_Cl/total_vol

vol_AlCl3_scale = ratio_vv_AlCl3*volume
vol_EMIm_Cl_scale = ratio_vv_EMIm_Cl*volume

mass_AlCl3_scale = vol_AlCl3_scale*density_AlCl3
mass_EMIm_Cl_scale = vol_EMIm_Cl_scale*density_EMIm_Cl

number_AlCl3_scale = (mass_AlCl3_scale/molar_mass_AlCl3)* 6.02214076e23
number_EMIm_Cl_scale = (mass_EMIm_Cl_scale/molar_mass_EMIm_Cl)* 6.02214076e23

print("AlCl3")
print(np.round(number_AlCl3_scale))
print("[EMIm]Cl")
print(np.round(number_EMIm_Cl_scale))

AlCl3
7.0
[EMIm]Cl
9.0


In [2]:
def generar_input_packmol(nombre_archivo, estructuras, limites_caja, tolerancia=2.0):
	"""
	Genera un archivo de entrada para Packmol.
	
	Args:
		nombre_archivo: Nombre del archivo de salida (.inp)
		estructuras: Lista de diccionarios con información de cada molécula.
						Cada diccionario debe tener:
						{'archivo': 'mol.pdb', 'numero': 10, 'restricciones': 'inside box 0 0 0 30 30 30'}
		limites_caja: Tupla (x_min, y_min, z_min, x_max, y_max, z_max) con los límites de la caja.
		tolerancia: Valor de tolerancia de distancia mínima entre moléculas.
	"""
	with open(nombre_archivo, 'w') as f:
		f.write(f"tolerance {tolerancia}\n")
		f.write("filetype xyz\n")  # Puede cambiarse a xyz si se usa ese formato
		f.write("output sistema_final.xyz\n\n")
		
		for estructura in estructuras:
			f.write("structure {}\n".format(estructura['archivo']))
			f.write("  number {}\n".format(estructura['numero']))
			f.write("  {}\n".format(estructura['restricciones']))
			f.write("end structure\n\n")
	
	print(f"Archivo {nombre_archivo}  correctamente.")

In [3]:
moleculas = [{"archivo":"estructuras/xyz/alcl3_opt.xyz", "numero": int(np.round(number_AlCl3_scale)), "restricciones": f"inside box 0 0 0 {lx-1.0} {ly-1.0} {lz-quitar}"},
			{"archivo":"estructuras/xyz/1-Ethyl-3-methylimidazolium_chloride_opt.xyz", "numero": int(np.round(number_EMIm_Cl_scale)), "restricciones": f"inside box 0 0 0 {lx-1.0} {ly-1.0} {lz-quitar}"}]

limites = (0,0,0,lx,ly,lz)
generar_input_packmol("mezcla.inp",moleculas, limites)

Archivo mezcla.inp  correctamente.


In [4]:
from ase import Atoms
from ase.io import read, write
from ase.visualize import view # Importa la herramienta para visualizar la estructura
mol = read("sistema_final.xyz")
#view(mol)

In [5]:
mol.cell = [[lx,0.0,0.0],[0.0,ly,0.0],[0.0,0.0,lz]]
mol.positions[:,0] += 1
mol.positions[:,1] += 1
mol.positions[:,2] += quitar
mol.center()
view(mol)

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/envs/genera...>

In [ ]:
# Genera un archivo de entrada para Quantum ESPRESSO con la configuración donde el hidrógeno está sobre el átomo de carbono.
write("pw.in", mol, format="espresso-in", 
	pseudopotentials={
	"C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",  # Especifica el pseudopotencial para carbono.
	"H": "H.pbe-kjpaw_psl.1.0.0.UPF",     # Especifica el pseudopotencial para hidrógeno.
	"N": "N.pbe-n-kjpaw_psl.1.0.0.UPF",
	"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF",
	"Cl": "Cl.pbe-n-kjpaw_psl.1.0.0.UPF"
}, input_data={
	"control": {
		"calculation": "md",
		"restart_mode": 'from_scratch',
		"nstep": 200,
		"tprnfor": True,
		"dt": 20,
		"prefix": "elec",    # Prefijo para los archivos de salida generados por Quantum ESPRESSO.
		"outdir": ".",             # Carpeta donde se guardarán los archivos temporales de Quantum ESPRESSO.
		"pseudo_dir": "/home/ebarquero/practicas/pseudo/"          # Carpeta donde se encuentran los pseudopotenciales.
	},
	"system": {
		"ecutwfc": 50,                    # Corte de energía para las funciones de onda (en Ry).
		"ecutrho": 330,                   # Corte de energía para la densidad de carga (en Ry).
		"occupations": "smearing",        # Usa un método de smearing para las ocupaciones electrónicas.
		"smearing": "mp",                 # Método de smearing: Methfessel-Paxton.
		"degauss": 0.02 ,                  # Ancho del smearing en Ry.
		"ibrav":0, 							# Define una celda generica
	},
	"electrons": {
		"conv_thr": 1e-8                  # Criterio de convergencia para las iteraciones electrónicas.
	},
	"ions":{
		"ion_dynamics": "verlet",
		"ion_temperature": "berendsen",
		"tempw": 300.0
	}
}, kpts = (1,1,1),koffset = (0,0,0))